In [1]:
import pandas as pd
import numpy as np
import altair as alt
from os.path import join

from constants import COLUMNS
from utils import read_latest_labs_df

In [2]:
df = read_latest_labs_df()
df

,siteid,loinc,days_since_positive,num_patients,mean_value,stdev_value
0,FMC,1920-8,-2,-1,27.0,1.000
1,FMC,1920-8,-1,4,41.2,23.567
2,FMC,1920-8,0,4,47.1,17.876
3,FMC,1920-8,1,38,59.2,40.150
4,FMC,731-0,1,-1,5.0,3.000
5,FMC,731-0,-1,1,11.2,63.567
6,FMC,731-0,0,7,41.1,27.876
7,FMC,731-0,1,22,12.2,30.150
8,FMC,1751-7,1,1,7.0,1.000
9,FMC,1751-7,0,-1,31.2,3.567


In [3]:
line = alt.Chart(df).mark_line().encode(
    x=alt.X(COLUMNS.DAYS_SINCE_POSITIVE, title="Days since positive", scale=alt.Scale(domain=[-6,8])),
    y=alt.Y(COLUMNS.MEAN_VALUE, title="Mean value"),
    color=alt.Color(COLUMNS.SITE_ID, title="Site")
)

point = alt.Chart(df).mark_point(filled=True).encode(
    x=alt.X(COLUMNS.DAYS_SINCE_POSITIVE, title="Days since positive"),
    y=alt.Y(COLUMNS.MEAN_VALUE, title="Mean value"),
    color=alt.Color(COLUMNS.SITE_ID, title="Site")
)


band = alt.Chart(df).mark_errorbar().encode(
    x=alt.X(COLUMNS.DAYS_SINCE_POSITIVE, title="Days since positive"),
    y=alt.Y(COLUMNS.MEAN_VALUE, title="Mean value"),
    yError=alt.YError(COLUMNS.STDEV_VALUE),
    color=alt.Color(COLUMNS.SITE_ID, title="Site")
)

(band + point + line).facet(
    column=alt.Column(COLUMNS.LOINC, title="LOINC")
)

alt.FacetChart(...)